In [1]:
import pandas as pd
from google.colab import files
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


In [ ]:
files.upload()

In [2]:
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
#df = pd.read_csv('Market_Basket_Optimisation.csv')
print(df.head(10))
print(df.shape)
print(df.columns)

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
5    536365     22752         SET 7 BABUSHKA NESTING BOXES         2   
6    536365     21730    GLASS STAR FROSTED T-LIGHT HOLDER         6   
7    536366     22633               HAND WARMER UNION JACK         6   
8    536366     22632            HAND WARMER RED POLKA DOT         6   
9    536367     84879        ASSORTED COLOUR BIRD ORNAMENT        32   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [3]:
# hay que verificar si hay valores faltantes en el dataframe
print(df.isnull().sum())

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [4]:
#%% vemos de qué países son las transacciones

print(df['Country'].unique())


['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Austria' 'Israel' 'Finland' 'Bahrain' 'Greece' 'Hong Kong'
 'Singapore' 'Lebanon' 'United Arab Emirates' 'Saudi Arabia'
 'Czech Republic' 'Canada' 'Unspecified' 'Brazil' 'USA'
 'European Community' 'Malta' 'RSA']


In [6]:
#%%   Eliminamos espacios en las descripciones y las filas donde no se identifica al comprador

df['Description'] = df['Description'].str.strip()
#df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
#df['InvoiceNo'] = df['InvoiceNo'].astype('str')


In [7]:
#%%  nos quedamos con los datos de Italia
# cada fila representa una factura y cada columna un producto
# creamos un nuevo dataframe con los productos comprados

basket = (df[df['Country']=="Italy"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket.drop('POSTAGE', inplace=True, axis=1)
print(basket.shape)
print(basket.head(5))

(55, 479)
Description  12 EGG HOUSE PAINTED WOOD  12 PENCILS TALL TUBE RED RETROSPOT  \
InvoiceNo                                                                    
537022                             0.0                                 0.0   
539752                             0.0                                 0.0   
541115                             0.0                                 0.0   
541703                             0.0                                 0.0   
542238                             0.0                                 0.0   

Description  12 PENCILS TALL TUBE SKULLS  12 PENCILS TALL TUBE WOODLAND  \
InvoiceNo                                                                 
537022                               0.0                            0.0   
539752                               0.0                            0.0   
541115                               0.0                            0.0   
541703                               0.0                            

In [8]:
# primero hay que preprocesar y dejarlo en formato one-hot encoder

basket_ok = pd.get_dummies(basket, sparse = True)
print(basket_ok.T.head(10))
print(basket_ok.columns)
print(basket_ok.shape)

InvoiceNo                           537022  539752  541115  541703  542238  \
Description                                                                  
12 EGG HOUSE PAINTED WOOD              0.0     0.0     0.0     0.0     0.0   
12 PENCILS TALL TUBE RED RETROSPOT     0.0     0.0     0.0     0.0     0.0   
12 PENCILS TALL TUBE SKULLS            0.0     0.0     0.0     0.0     0.0   
12 PENCILS TALL TUBE WOODLAND          0.0     0.0     0.0     0.0     0.0   
16 PIECE CUTLERY SET PANTRY DESIGN     0.0     0.0     0.0     0.0     0.0   
20 DOLLY PEGS RETROSPOT                0.0     0.0     0.0     0.0     0.0   
3 GARDENIA MORRIS BOXED CANDLES        4.0     0.0     0.0     0.0     0.0   
3 ROSE MORRIS BOXED CANDLES            4.0     0.0     0.0     0.0     0.0   
3 STRIPEY MICE FELTCRAFT               0.0     0.0     0.0     0.0     0.0   
3 TIER CAKE TIN RED AND CREAM          0.0     0.0     0.0     0.0     0.0   

InvoiceNo                           544399  545664  546875  547

In [9]:
# veamos los items más frecuentes, le ponemos un soporte mínimo de 10%
basket_ok[basket_ok >= 1.] = 1.
basket_ok[basket_ok < 1.] = 0.

print(~basket_ok.T.isin([0., 1.]).any())
#print(basket_ok.head())

InvoiceNo
537022     False
539752     False
541115     False
541703     False
542238     False
544399     False
545664     False
546875     False
547914     False
547931     False
549233     False
550606     False
555107     False
555843     False
556119     False
561259     False
562424     False
562528     False
562922     False
564476     False
567478     False
570919     False
571670     False
572457     False
572695     False
572934     False
573365     False
573508     False
573768     False
574022     False
576907     False
577178     False
577316     False
577609     False
577809     False
578078     False
578108     False
580746     False
C537232    False
C540129    False
C542273    False
C543743    False
C546501    False
C547930    False
C558439    False
C562423    False
C562425    False
C562969    False
C569949    False
C572761    False
C576199    False
C576216    False
C578239    False
C579785    False
C579864    False
dtype: bool


In [10]:
frequent_itemsets = apriori(basket_ok, min_support=0.05, use_colnames=True)


In [11]:
print(frequent_itemsets.head(10))
print(frequent_itemsets.tail(10))
print(frequent_itemsets.columns)
print(frequent_itemsets.shape)

    support                          itemsets
0  0.072727           (ABC TREASURE BOOK BOX)
1  0.054545       (ADULT APRON APPLE DELIGHT)
2  0.072727    (BAKING SET 9 PIECE RETROSPOT)
3  0.109091     (BREAD BIN DINER STYLE IVORY)
4  0.054545      (BREAD BIN DINER STYLE PINK)
5  0.109091   (CHILDRENS APRON APPLES DESIGN)
6  0.090909                 (DOORMAT AIRMAIL)
7  0.072727  (DOORMAT SPOTTY HOME SWEET HOME)
8  0.109091              (DOORMAT UNION FLAG)
9  0.109091     (DOORMAT WELCOME TO OUR HOME)
      support                                           itemsets
258  0.054545  (SET OF 20 KIDS COOKIE CUTTERS, RETROSPOT TEA ...
259  0.054545  (DOORMAT UNION FLAG, BREAD BIN DINER STYLE IVO...
260  0.054545  (JUMBO BAG WOODLAND ANIMALS, TOY TIDY PINK POL...
261  0.054545  (TOY TIDY SPACEBOY, JUMBO BAG WOODLAND ANIMALS...
262  0.054545  (TOY TIDY SPACEBOY, JUMBO BAG WOODLAND ANIMALS...
263  0.054545  (TOY TIDY SPACEBOY, TOY TIDY PINK POLKADOT, RE...
264  0.054545  (TOY TIDY SPACEBOY, JUMB

In [12]:
rules = association_rules(frequent_itemsets, 
                          metric="confidence", min_threshold=0.5)
print(rules.head())

                     antecedents                    consequents  \
0        (ABC TREASURE BOOK BOX)  (RED RETROSPOT CHARLOTTE BAG)   
1  (RED RETROSPOT CHARLOTTE BAG)        (ABC TREASURE BOOK BOX)   
2        (ABC TREASURE BOOK BOX)       (WOODLAND CHARLOTTE BAG)   
3       (WOODLAND CHARLOTTE BAG)        (ABC TREASURE BOOK BOX)   
4           (DOORMAT UNION FLAG)  (BREAD BIN DINER STYLE IVORY)   

   antecedent support  consequent support   support  confidence       lift  \
0            0.072727            0.054545  0.054545    0.750000  13.750000   
1            0.054545            0.072727  0.054545    1.000000  13.750000   
2            0.072727            0.054545  0.054545    0.750000  13.750000   
3            0.054545            0.072727  0.054545    1.000000  13.750000   
4            0.109091            0.109091  0.072727    0.666667   6.111111   

   leverage  conviction  
0  0.050579    3.781818  
1  0.050579         inf  
2  0.050579    3.781818  
3  0.050579         inf 

In [13]:
rules.tail()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1315,(JUMBO BAG WOODLAND ANIMALS),"(TOY TIDY SPACEBOY, TOY TIDY PINK POLKADOT, RE...",0.072727,0.054545,0.054545,0.75,13.750000,0.050579,3.781818
1316,(TOY TIDY PINK POLKADOT),"(TOY TIDY SPACEBOY, JUMBO BAG WOODLAND ANIMALS...",0.090909,0.054545,0.054545,0.60,11.000000,0.049587,2.363636
1317,(RECYCLING BAG RETROSPOT),"(TOY TIDY SPACEBOY, JUMBO BAG WOODLAND ANIMALS...",0.109091,0.054545,0.054545,0.50,9.166667,0.048595,1.890909
1318,(CHILDRENS APRON APPLES DESIGN),"(TOY TIDY SPACEBOY, JUMBO BAG WOODLAND ANIMALS...",0.109091,0.054545,0.054545,0.50,9.166667,0.048595,1.890909
1319,(JUMBO BAG TOYS),"(TOY TIDY SPACEBOY, JUMBO BAG WOODLAND ANIMALS...",0.054545,0.054545,0.054545,1.00,18.333333,0.051570,inf


In [14]:
rules.shape

(1320, 9)

In [ ]:
print(rules.columns)
print(rules.shape)

Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction'],
      dtype='object')
(1320, 9)


In [16]:
# veamos si cambia usando otra métrica
 
rules2 = association_rules(frequent_itemsets, metric="lift", min_threshold=5)
print(rules2.head())
print(rules2.shape)



                     antecedents                    consequents  \
0        (ABC TREASURE BOOK BOX)  (RED RETROSPOT CHARLOTTE BAG)   
1  (RED RETROSPOT CHARLOTTE BAG)        (ABC TREASURE BOOK BOX)   
2        (ABC TREASURE BOOK BOX)       (WOODLAND CHARLOTTE BAG)   
3       (WOODLAND CHARLOTTE BAG)        (ABC TREASURE BOOK BOX)   
4           (DOORMAT UNION FLAG)  (BREAD BIN DINER STYLE IVORY)   

   antecedent support  consequent support   support  confidence       lift  \
0            0.072727            0.054545  0.054545    0.750000  13.750000   
1            0.054545            0.072727  0.054545    1.000000  13.750000   
2            0.072727            0.054545  0.054545    0.750000  13.750000   
3            0.054545            0.072727  0.054545    1.000000  13.750000   
4            0.109091            0.109091  0.072727    0.666667   6.111111   

   leverage  conviction  
0  0.050579    3.781818  
1  0.050579         inf  
2  0.050579    3.781818  
3  0.050579         inf 